In [1]:
import os
%pwd

'c:\\Users\\karba\\OneDrive\\Desktop\\MLOPS\\TextSummerizer\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\karba\\OneDrive\\Desktop\\MLOPS\\TextSummerizer'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int


In [5]:
from src.TextSummerizer.constants import *
from src.TextSummerizer.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            eval_strategy = params.eval_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )
        return model_trainer_config



In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk

[2024-10-29 16:35:32,908: INFO: config: PyTorch version 2.5.1+cu124 available.]


In [8]:
"cuda" if torch.cuda.is_available() else "cpu"

'cuda'

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        #loading the data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            eval_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 
        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))




In [9]:
# !pip install --upgrade accelerate
# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate

In [10]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()

[2024-10-29 16:35:41,170: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-29 16:35:41,177: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-29 16:35:41,179: INFO: common: created directory at: artifacts]
[2024-10-29 16:35:41,180: INFO: common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\karba\AppData\Local\Temp\ipykernel_13920\2273714221.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args=trainer_args,


  0%|          | 0/51 [00:00<?, ?it/s]

{'loss': 48.3645, 'grad_norm': 192.98477172851562, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}
{'loss': 53.1849, 'grad_norm': 230.52401733398438, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.39}
{'loss': 49.6467, 'grad_norm': 1295.934814453125, 'learning_rate': 3e-06, 'epoch': 0.59}
{'loss': 45.6968, 'grad_norm': 198.67820739746094, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.78}
{'loss': 44.7179, 'grad_norm': 267.02728271484375, 'learning_rate': 5e-06, 'epoch': 0.98}


c:\Users\karba\OneDrive\Desktop\MLOPS\TextSummerizer\venv\lib\site-packages\transformers\modeling_utils.py:2816: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


{'train_runtime': 652.2944, 'train_samples_per_second': 1.256, 'train_steps_per_second': 0.078, 'train_loss': 48.17260368197572, 'epoch': 1.0}
